In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization,Activation, MaxPool2D, Concatenate
from tensorflow.keras.optimizers import Adam
from PIL import Image 
import numpy as np
import re
import os 
import glob 
import random
import logging
from tensorflow.keras import backend as K
import math

In [2]:
data_dir = "/classifier_data/"
barcodes = os.listdir(data_dir)
print(len(barcodes))

757


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = (224, 224)
batch_size = 32 * 4 
num_classes = 757
data_directory = "/classifier_data/"

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    data_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=42,
    subset='training'
)

Found 225554 images belonging to 757 classes.


In [6]:
validation_generator = train_datagen.flow_from_directory(
    data_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    seed=42,
    subset='validation'
)

Found 24744 images belonging to 757 classes.


In [7]:
def build_model(input_shape, num_classes):
    base_model = tf.keras.applications.EfficientNetB5(include_top=False, weights="imagenet")
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = base_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)  
    x = tf.keras.layers.Dense(1024, activation='relu')(x)  
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)  
    model = tf.keras.Model(inputs, outputs)
    return model

In [8]:
strategy = tf.distribute.MirroredStrategy()
NUM_REPLICAS = strategy.num_replicas_in_sync
BATCH_PER_REPLICAS = 32
GLOBAL_BATCH = BATCH_PER_REPLICAS * NUM_REPLICAS
AUTOTUNE = tf.data.experimental.AUTOTUNE

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [9]:
with strategy.scope():
    model = build_model((128,128,3), 757)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [10]:
history = model.fit(train_generator, epochs=10)

Epoch 1/10
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:batch_all_reduce: 508 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 508 all-reduces with algorithm = nccl, num_packs = 1
1763/1763 [==============================] - 1528s 867ms/step - loss: 1.1616 - accuracy: 0.6954
Epoch 2/10
  57/1763 [..............................] - ETA: 26:31 - loss: 0.3763 - accuracy: 0.8805

KeyboardInterrupt: 

In [ ]:
model.save_weights("model.h5")